In [ ]:
import os
from IPython.display import IFrame, display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets

def on_change(change): # si attiva quando cambio scelta dell'elemento da selezionare
    global attach
    if change['type'] == 'change' and change['name'] == 'value':
        #print ("Stai scegliendo %s" % change['new'])
        attach = change['new']
        return attach

def on_button_update(_):
    clear_output()
    #display(Javascript('''var c = IPython.notebook.get_selected_index();
                       #IPython.notebook.execute_cells([c])'''))
    allegato()

def open_attachment(attach_name):
    path_attachments = "allegati"
    path_file=str(path_attachments + "/" + attach_name)
    ext = os.path.splitext(path_file)[-1].lower()
    #print(path_file)
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        #file = open(path_file, "rb")
        display(Markdown("![alt text](" + path_file + ")"))
        delete = widgets.Button(description='Rimuovi allegato')
        delete.on_click(on_button_delete)
        display(delete)

        #image = file.read()
        #w=widgets.Image(
        #    value=image,
        #    format=ext.replace('.', ''),
        #    width=300,
        #    height=400,
        #)
        #display(w)

def on_button_confirm(_):
    global attach
    clear_output()
    print('Allegato: ' + str(attach))
    open_attachment(attach)

def allegato():
    path_attachments = "allegati"
    global attach # allegato attuale scelto
    update = widgets.Button(description='Aggiorna') #bottone visualizzato in ogni caso
    delete = widgets.Button(description='Rimuovi allegato')
    attach_list = os.listdir(path_attachments)
    if 'ck_points' in attach_list:
        attach_list.remove('ck_points')

    if(attach_list): #si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        confirm = widgets.Button(description='Conferma')
        chosen_attach_wid = widgets.Dropdown(
            options = attach_list,
            value = attach_list[0],
            description = 'Allegati:',
            disabled = False,
        )
        attach = attach_list[0]
        chosen_attach_wid.observe(on_change)
        update.on_click(on_button_update)
        confirm.on_click(on_button_confirm)
        delete.on_click(on_button_delete)
        all_buttons = [update, chosen_attach_wid, confirm, delete]
        display(widgets.HBox(all_buttons))
    else:
        print(f"ATTENZIONE: non sono presenti file nella cartella allegati.") #Ti ricordiamo che la cartella utile per gli allegati da te prodotti per questo esercizio in modo_libero di svolgimento è la cartella @path_ex_folder@/modo_libero/allegati/")
        update.on_click(on_button_update)
        delete.on_click(on_button_delete)
        all_buttons = [update, delete]
        display(widgets.HBox(all_buttons))

def on_button_delete(_):
    #delete_above_cell()
    delete_this_cell()

def delete_above_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c-1).metadata.editable = true;
                                IPython.notebook.get_cell(c-1).metadata.deletable = true;
                                IPython.notebook.delete_cell([c-1]);'''))
def delete_this_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c).metadata.editable = true;
                                IPython.notebook.get_cell(c).metadata.deletable = true;
                                IPython.notebook.delete_cell([c]);'''))

def on_button_raw_attach(_):
    add_attach_cell_din('allegato()')

def on_button_md(_):
    add_md_cell_din()

def on_button_code(_):
    add_code_cell_din()

def on_button_raw(_):
    add_raw_cell_din()

def add_raw_cell_din():
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var idx = IPython.notebook.get_selected_index();
                       var c = IPython.notebook.insert_cell_at_index("raw", idx);
    c.set_text('');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_attach_cell_din(code = ''):
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.editable = false;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_code_cell_din(code = ''):
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id)
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_md_cell_din():
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_cell(code='', position='below', celltype='markdown', is_execute = False):
    """Create a cell in the IPython Notebook.
    code: unicode, Code to fill the new cell with.
    celltype: unicode, Type of cells "code" or "markdown".
    position: unicode, Where to put the cell "below" or "at_bottom"
    is_execute: boolean, To decide if the cell is executed after creation
    """

    # Create a unique id based on epoch time
    display_id = int(time.time()*1e9)

    if is_execute:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        basis.execute();
        """.format(position, celltype, " ")),display_id=display_id)

    else:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        """.format(position, celltype, " ")),display_id=display_id)


    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)

def code_button_delete():
    button_delete = widgets.Button(description="Rimuovi l'allegato", tooltip="Seleziona la cella e clicca su Elimina")
    button_delete.on_click(on_button_delete)
    #print("Vuoi eliminare la cella selezionata?")
    display(button_delete)

def loader_main():
    button_raw_attach = widgets.Button(description="(Cella per Allegato)", tooltip="Collega un file dalla cartella attachments")
    button_md = widgets.Button(description="(Cella Markdown)", abstooltip="Aggiungi una cella per scrivere del testo in Markdown")
    button_code = widgets.Button(description="(Cella Codice)", tooltip="Aggiungi una cella per scrivere del codice in Python")
    button_raw = widgets.Button(description="(Cella Testo ASCII)", tooltip="Aggiungi una cella per scrivere del testo libero (SCONSIGLIATA: righe troppo lunghe potrebbero fuoriuscire nella rendition, usala solo se non riesci a controllare il Markdown)")
    button_raw_attach.on_click(on_button_raw_attach)
    button_md.on_click(on_button_md)
    button_code.on_click(on_button_code)
    button_raw.on_click(on_button_raw)

    all_buttons = [button_code, button_md, button_raw, button_raw_attach]
    display(widgets.HBox(all_buttons))


# ESERCIZIO: Problema dello zaino #


In [ ]:
import os
from IPython.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame
import numpy as np


In [ ]:
    from tabulate import tabulate
    import copy
    n_tasks = 10;
    arr_point = [-1] * n_tasks;

In [ ]:

    labels= ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N']
    costs = [15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7]
    vals = [50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22]
    CapacityMax= 36
    

In [ ]:
# RTal python library for Jupyter Notebook usage
# Authors: Michele Albrigo, Romeo Rizzi
# Requirements: websocket, json

# to install websocket inside a Jupyter notebook use:
# import sys
# !{sys.executable} -m pip install websocket-client

from sys import stderr
import websocket
from IPython.display import display, HTML
import json
import ruamel.yaml
import os

RTAL_PRIVATE_ACCESS_TOKEN="id123456_VR123456_gCj0c44poMqbW5x"
PATH_EDITABLE_SETTINGS = os.getcwd()
while os.path.isdir(PATH_EDITABLE_SETTINGS):
    if os.path.isfile(os.path.join(PATH_EDITABLE_SETTINGS,'settings','settings.yaml')):
        break
    PATH_EDITABLE_SETTINGS = os.path.join(PATH_EDITABLE_SETTINGS,'..')
    SETTINGS_FILE_FULLNAME=os.path.join(PATH_EDITABLE_SETTINGS,'settings','settings.yaml') 
with open(SETTINGS_FILE_FULLNAME,"r") as stream:
     settings = ruamel.yaml.safe_load(stream)


def info():
    print("These are your current dynamic settings, as set in the file",end="\n   ")
    print(SETTINGS_FILE_FULLNAME)
    for k,v in settings.items():
        print(f"{k}: {v}")
    hint = """
To see all the info you can get enter and run this piece of code:
   vars = [k for k in locals().keys() if k[0] != '_']
   vars.sort()
   print('\\n'.join(iter(vars)))"""
    print(hint)

def printif(category_of_interest,msg):
    if category_of_interest in settings['RTAL_INTERFACE_VERBOSITY']:
        print(msg)

    
def rtal_list(rtal_URL):
    """lists all problems served by the rtald server
    Parameters:
    - rtal_URL: rtal server address"""
    try:
        rtalws = websocket.create_connection(rtal_URL)
        rtalws.send('{"Handshake":{"magic":"rtal","version":2}}')
        printif('TALight_protocol','TALight return code:' + rtalws.recv())
        rtalws.send('{"MetaList":{}}')
        listjson = json.loads(rtalws.recv())
        printif('TALight_protocol',list((listjson['MetaList'])['meta'].keys()))
        rtalws.close()
    except ConnectionRefusedError:
        printif('TALight_protocol_signals_problem',f'Could not connect to RTAL server {rtal_URL}')

def rtal_connect(rtal_URL, rtalproblem, rtalservice, rtalargs_dict, rtaltoken=None, output_files_local_folder=None):
    """sends a TALight connect request (problem, service, args)
    Parameters:
    - rtal_URL: rtal server address
    - rtalproblem: name of the TALight problem offering the solution checking and oracle services
    - rtalservice: name of the checking TALight service of interest
    - rtalargs_dict: dictionary with the arguments to be sent to the TALight service
    - output_files_local_folder (optional): the name of the local folder where TALight could store files like e.g. a certificate for a good submission received 
    - rtaltoken (optional): the student access token so that logs are stored on the server or even just to access the service"""
    rtalfiles= 'RO_knapsack_out.txt'
    """NOTA [MISTERO DELLA FEDE]: se ora da quì dentro faccio un qualsiasi print (anche se su stderr) prima di entrare nel try e farlo come da lì, allora le cose si rompono e chi ha chiamato la funzione lamenta di non aver ricevuto risposta attribuendolo ad invalid TOKEN.
       Ad esempio se faccio:
           print("\n\n\n" f"rtalargs_dict={rtalargs_dict}" "\n\n\n")
       Poi successivamente nelle richieste nel notebook esprime:
           tmp_first_reply={'ConnectBegin': {'status': {'Err': 'Invalid token'}}}
Ho incontrato errori nel richiedere a TALight il servizio check del problema RO_knapsack
Causa Errore: Invalid token
Spiacenti, il feedback da TALight non ha potuto essere prodotto.
       e la situazione non si ripristina a corretta nemmeno se reverto il cambiamento, il suo effetto resta permanente anche se rieseguo tutte le celle.
    """
    args_string = ""
    for key in rtalargs_dict:
        args_string += f'"{key}":"{rtalargs_dict[key]}", '
    args_string = args_string[:-2]
    try:
        rtalws = websocket.create_connection(rtal_URL)
        rtalws.send('{"Handshake":{"magic":"rtal","version":2}}')
        connection_request_outcome = rtalws.recv()
        printif('TALight_protocol','TALight return code:' + connection_request_outcome)
        if rtaltoken == None:
            if output_files_local_folder == None:
                printif('TALight_protocol',"sending 1: " + '{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"files":["'+rtalfiles+'"]}}')
                rtalws.send('{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"files":["'+rtalfiles+'"]}}')
            else:
                printif('TALight_protocol',"sending 2: " + '{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"files":["'+rtalfiles+'"]}}')
                rtalws.send('{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"files":["'+rtalfiles+'"]}}')
        else:
            if output_files_local_folder == None:
                printif('TALight_protocol',"sending 3: " + '{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"token":"'+rtaltoken+'","files":["'+rtalfiles+'"]}}')
                rtalws.send('{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"token":"'+rtaltoken+'","files":["'+rtalfiles+'"]}}')
            else:
                printif('TALight_protocol',"sending 4: " + '{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"token":"'+rtaltoken+'","files":["'+rtalfiles+'"]}}')
                rtalws.send('{"ConnectBegin":{"problem":"'+rtalproblem+'","service":"'+rtalservice+'","args":{'+args_string+'},"tty":false,"token":"'+rtaltoken+'","files":["'+rtalfiles+'"]}}')
        #tmp_first_reply = ruamel.yaml.safe_load(rtalws.recv())
        tmp_first_reply = json.loads(rtalws.recv())
        printif('TALight_protocol',f'tmp_first_reply={tmp_first_reply}')
        if "Err" in tmp_first_reply["ConnectBegin"]["status"]:
            printif('TALight_protocol_signals_problem',f'Ho incontrato errori nel richiedere a TALight il servizio {rtalservice} del problema {rtalproblem}')
            printif('TALight_protocol_signals_problem',"Causa Errore: " + tmp_first_reply["ConnectBegin"]["status"]["Err"])
            printif('TALight_protocol_signals_problem',"Spiacenti, il feedback da TALight non ha potuto essere prodotto.")
            return None
        else:
            connection_request_outcome = rtalws.recv()
            printif('TALight_protocol','TALight connection status:' + connection_request_outcome)
            TAL_service_printout = rtalws.recv().decode('UTF-8')
            printif('TALight_protocol','TALight service printout:' + TAL_service_printout)
            feedback_dict={'feedback_string':TAL_service_printout}
            if '"as_yaml_with_points":"1"' in args_string:
                try:
                    feedback_dict=eval(TAL_service_printout)
                except:
                    pass
                
            if '"color_implementation":"html"' in args_string:
                feedback_dict['feedback_string']=HTML(feedback_dict['feedback_string'])
            return feedback_dict
            if output_files_local_folder != None:
               with open(output_files_local_folder, 'w') as output_file:
                    output_file.write(TAL_service_printout)
                    output_file.close()
            rtalws.close()
    except ConnectionRefusedError:
        printif('TALight_protocol_signals_problem',f'Could not connect to RTAL server {rtal_URL}')
        


# Test TALight list command (check which problems are currently served. The purpouse is just to check if you reach the server):

if settings['DEBUG_RTAL_AT_SET_UP']:
    for rtal_URL_name in settings['RTAL_URLS']:
        URL = settings[rtal_URL_name]
        print("\n"f"List of TAL problems served on server {rtal_URL_name} [{URL}]:")
        rtal_list(URL)

# Example on how to lay down the data to prepare for an rtal_connect call for a specific problem when testing its correct interface as for the TALight server:

task_dict={'task': 1, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0, 'with_positive_enforcement':'1', 'with_notes':'1', 'as_yaml_with_points':'1','color_implementation':'html','with_output_files':'0'}
input_data_assigned={'labels': ['A', 'B', 'C', 'D', 'E'], 'costs': [2, 3, 4, 5, 6], 'vals': [13, 17, 19, 30, 101], 'Knapsack_Capacity': 5, 'forced_out': [], 'forced_in': [], 'partialDPtable': []}
alias_dict={'opt_sol69':'opt_sol','opt_val69':'opt_val'}
answer_dict={'opt_sol69':['A','B'],'opt_val69':30}
rtalargs_dict = task_dict
rtalargs_dict['input_data_assigned'] = input_data_assigned
rtalargs_dict['alias_dict'] = alias_dict
rtalargs_dict['answer_dict'] = answer_dict

def monitor_what_submitted_to_rtald(problem_name, rtalargs_dict):
    if settings['MONITOR_CALLS_TO_RTAL_CHECKERS']:
        command_line = f"rtal connect {problem_name} check  "
        for key in rtalargs_dict:
            command_line += f'-a {key}="{rtalargs_dict[key]}"  '
        print("\n\n\n" + "*-------"*5)
        print("SICCOME PARLIAMO DIRETTAMENTE AD RTALD DOBBIAMO PASSARGLI LA LISTA COMPLETA DEGLI ARGOMENTI (IL DEFAULTING NON RISOLVE). QUESTO MESSAGGIO SERVE PER TENERTI CONSAPEVOLE DI COSA ESCE E CONSENTIRTI DI REPLICARLO AGEVOLMENTE A RIGA DI COMANDO PER TESTARLO IN CASO DI PROBLEMI\n\n" f"Gestire rtalargs_dict={rtalargs_dict}" "\n\ncorrisponde alla riga seguente di comando:\n   " f"$ {command_line}")
        print("*-------"*5 + "*\n\n\n")


monitor_what_submitted_to_rtald("RO_knapsack", rtalargs_dict)

# Example call with TOKEN to an rtal problem service meant to play as a checker:
if settings['DEBUG_RTAL_AT_SET_UP']:
    for rtal_URL_name in settings['RTAL_URLS']:
        rtal_URL = settings[rtal_URL_name]
        print("\n"f"Now calling server {rtal_URL_name} [{rtal_URL}]:")
        feedback_dict = rtal_connect(rtal_URL, 'RO_knapsack', 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        print("\n"f"Feedback from server {rtal_URL_name} [{rtal_URL}]:")
        if feedback_dict != None:
            display(feedback_dict['feedback_string'])
        else:
            print("\n"f"The Feedback received from the server {rtal_URL_name} is EMPTY")
            

# Example call without TOKEN to an rtal problem service meant to play as a checker:
if settings['DEBUG_RTAL_AT_SET_UP']:
    for rtal_URL_name in settings['RTAL_URLS']:
        rtal_URL = settings[rtal_URL_name]
        feedback_dict = rtal_connect(rtal_URL, 'RO_knapsack', 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=None)
        print(f"Feedback from server {rtal_URL_name} [{rtal_URL}]:")
        if feedback_dict != None:
            display(feedback_dict['feedback_string'])
        else:
            print("The Feedback received from the server is EMPTY")


# Example call with to an rtal problem checker service with files saved in local:
if settings['DEBUG_RTAL_AT_SET_UP']:
    print("files saved in local: -NOT YET IMPLEMENTED-")

# Example call to an rtal problem service meant to play as an oracle:
if settings['DEBUG_RTAL_AT_SET_UP']:
    print("files saved in local: -NOT YET COME INTO THE GAME-")



In [ ]:
# General Instantiation for the Verifiers# library for the standardization of the feedback coming from the checkers placed within the jupyter notebook
# it offers the same interface as a twin library for the RO-problems aunder TALight. In this way it better supports the in site integration of the code of the checking services of those TALight problems built to support exercises to be deployed within jupyther notebooks.

from sys import stderr
from typing import Optional, List, Dict, Callable
import termcolor 
from ansi2html import Ansi2HTMLConverter
ansi2html = Ansi2HTMLConverter(inline = True)

class std_eval_feedback:
    def __init__(self, ENV):
        self.color_implementation = ENV["color_implementation"]
        self.new_line = '\n' if ENV["color_implementation"] == "ANSI" else '\\n'
        self.with_positive_enforcement = ENV["with_positive_enforcement"]
        self.with_notes = ENV["with_notes"]
        self.task_number = ENV["task"]
        self.pt_tot = ENV["pt_tot"]
        self.pt_formato_OK = ENV["pt_formato_OK"]
        self.pt_feasibility_OK = ENV["pt_feasibility_OK"]
        self.pt_consistency_OK = ENV["pt_consistency_OK"]
        self.feedback_so_far = ""
        self.completed_feedback = False

    def feedback_append(self, last_feedback_msg:str):
        self.feedback_so_far += last_feedback_msg
        #print(f"current feedback={self.feedback_so_far}",file=stderr)
    
    def colored(self, msg_text, *msg_rendering):
        if self.color_implementation == None:
            return msg_text
        if len(msg_rendering) == 0:
            ANSI_msg = msg_text
        else:
            if type(msg_rendering[-1]) == list:
                msg_style = msg_rendering[-1]
                msg_colors = msg_rendering[:-1]
            else:
                msg_style = []
                msg_colors = msg_rendering
            ANSI_msg = termcolor.colored(msg_text, *msg_colors, attrs=msg_style)
        if self.color_implementation == 'ANSI':
            colored_msg = ANSI_msg
        elif self.color_implementation == 'html':
            colored_msg = ansi2html.convert(ANSI_msg.replace(">", "&gt;").replace("<", "&lt;"), full=False).replace(self.new_line, "\n<br/>")
        else:
            assert self.color_implementation == None
        return colored_msg

    def evaluation_format(self, explanation, pt_safe:Optional[int] = None,pt_out:Optional[int] = None):
        pt_maybe = self.pt_tot-(pt_safe if pt_safe != None else 0)-(pt_out if pt_out != None else 0)
        index_pt=self.task_number-1
        arr_point[index_pt]=pt_safe
        file = open("points.txt", "w")
        file.write(str(arr_point))
        file.close()
        self.summary_of_scores = self.colored(f"{self.new_line}Totalizzi ", ["bold"]) + \
                               self.colored(f"[punti sicuri: {pt_safe}]", "green", ["bold"]) + ", " + \
                               self.colored(f"[punti aggiuntivi possibili: {pt_maybe}]", "blue", ["bold"]) + ", " + \
                               self.colored(f"[punti fuori portata: {pt_out}]", "red", ["bold"])
        self.last_explanation = self.colored(f"{self.new_line}Spiegazione: ", "cyan", ["bold"]) + explanation + self.colored(f"{self.new_line}")
        self.feedback_append(self.summary_of_scores)
        self.feedback_append(self.last_explanation)
        if pt_safe is None:
            pt_safe = 0
        if pt_out is None:
            pt_out = 0
        self.completed_feedback = {'pt_safe':pt_safe,'pt_maybe':pt_maybe,'pt_out':pt_out,'pt_available':self.pt_tot,'feedback_string':self.feedback_so_far,'summary-of-scores':self.summary_of_scores,'last_explanation':self.last_explanation}

   
    class goal:
        def __init__(self, goal_std_name, ad_hoc_name, answer_object):
            self.std_name = goal_std_name
            self.alias = ad_hoc_name
            self.answ = answer_object
        def __repr__(self):
            return f'Object goal(self.std_name={self.std_name}, self.alias={self.alias}, self.answ={self.answ})'
        
    def load(self, long_answer_dict:Dict):
        goals = {}
        for std_name,obj_with_alias in long_answer_dict.items():
            goals[std_name] = self.goal(std_name, ad_hoc_name=obj_with_alias[1], answer_object=obj_with_alias[0])
        return goals

    
    def voice_NO(self, voice:str, explanation:str):
        self.feedback_append(f"• {voice}: "+self.colored(f"NO.", "red", ["bold"])+self.colored(f".", "red")+self.colored(f".", "magenta") + self.colored(f".(motivo: ", "magenta", ["bold"]) + explanation + self.colored(")", "magenta", ["bold"]) + self.colored(self.new_line))

    def voice_OK(self, voice:str, positive_enforcement:str, note:str):
        self.feedback_append(f"• {voice}: "+self.colored(f"OK", "green", ["bold"]))
        if self.with_positive_enforcement:
            self.feedback_append(self.colored(".", "green", ["bold"])+self.colored(f"..(infatti: ", "green") + positive_enforcement + self.colored(")", "green"))
        if self.with_notes:
            self.feedback_append(self.colored(".", "green", ["bold"])+self.colored(f".", "green")+self.colored(f".", "cyan") + self.colored(f".(nota: ", "cyan", ["bold"]) + note + self.colored(")", "cyan", ["bold"]))
        self.feedback_append(self.colored(self.new_line))
        
    def format_NO(self, goal, explanation):
        self.voice_NO(f"Formato di `{goal.alias}`", explanation)
        self.evaluation_format(explanation, pt_safe=None,pt_out=self.pt_tot)
        return False
        
    def format_OK(self, goal, positive_enforcement, note):
        self.voice_OK(f"Formato di `{goal.alias}`", positive_enforcement, note)
        
    def feasibility_NO(self, goal, explanation):
        self.voice_NO(f"Ammissibilità di `{goal.alias}`", explanation)
        self.evaluation_format(explanation, pt_safe=self.pt_formato_OK,pt_out=self.pt_tot-self.pt_formato_OK)
        return False
        
    def feasibility_OK(self, goal, positive_enforcement, note):
        self.voice_OK(f"Ammissibilità di `{goal.alias}`", positive_enforcement, note)
        
    def consistency_NO(self, goals, explanation):
        self.voice_NO(f"Consistenza tra `{'` e `'.join([goal_std_name for goal_std_name in goals])}`", explanation)
        self.evaluation_format(explanation, pt_safe=self.pt_formato_OK+self.pt_feasibility_OK,pt_out=self.pt_tot-self.pt_formato_OK-self.pt_feasibility_OK)
        return False
        
    def consistency_OK(self, goals, positive_enforcement, note):
        self.voice_OK(f"Consistenza tra `{'` e `'.join([goal_std_name for goal_std_name in goals])}`", positive_enforcement, note)

            
    def feedback_when_all_checks_passed(self):
        self.evaluation_format(f"Quanto sottomesso{'' if self.task_number < 0 else ' per la Richiesta '+str(self.task_number)} ha superato tutti i miei controlli. Ovviamente in sede di esame non posso esprimermi sull'ottimalità di valori e di soluzioni immesse. Il mio controllo e supporto si è limitato alla compatibilità di formato, all'ammissibilità, e alla consistenza dei dati immessi.", pt_safe=self.pt_formato_OK + self.pt_feasibility_OK + self.pt_consistency_OK,pt_out=0)
        return self.completed_feedback

# defining the base class verify_submission_gen (valid for any RO_* problem)

from sys import stderr
from typing import Optional, List, Dict, Callable
from types import SimpleNamespace

class verify_submission_gen:
    def __init__(self, SEF,input_data_assigned:Dict, long_answer_dict:Dict):
        self.I = SimpleNamespace(**input_data_assigned)
        self.goals = SEF.load(long_answer_dict)
        self.long_answer_dict = long_answer_dict

    def verify_format(self, SEF):
        """In realtà questo tipo di controllo è attualmente demandato alla funzione check_and_standardization_of_request_answer_consistency  del modluo RO_std_io_lib.py
           Preferisco tuttavia richiamarlo quì perchè:
              1. non è del tutto chiaro (anche in relazione all'esperienza d'uso che avviene dentro i fogli Jupyther, il messaggio prodotto dalla funzione check_and_standardization_of_request_answer_consistenc  al momento è troppo severo, e d'altronde sono messaggi del modulo multilanguage che in questo momento non intedo toccare, ma posso duplicare il codice di quella parte e forse quella è la soluzione migliore, ma a quel punto la domanda è perchè non portare quì il controllo di tipo  togliendolo da  check_and_standardization_of_request_answer_consisten) quale possa essere il suo miglior collocamento
              2. come promemoria che vogliamo inventarci modi per portare a fattor comune controlli da problemi diversi.
           Tuttavia, per quanto riguarda 2, la via importante è che in questo modulo si definiscano delle funzioni general purpose per fare controlli sufficientemente generali (per definizione di un qualche linguaggio minimale e/oppure passando ad esse funzioni che fanno il controllo). Ma certo trovare delle buone soluzioni (con intelligenti compromessi) avrebbe il doppio vantaggio di sgravare il lavoro del problem maker e di offrire un'esperienza d'uso più standardizzata allo studente/problem solver.
        """
        for g in self.goals:
            if self.long_answer_dict[g] == 'int':
                if type(g.answ) != int:
                    return SEF.format_NO(g, f"Come `{g.alias}` hai immesso `{g.answ}` dove era invece richiesto di immettere un intero.")
                SEF.format_OK(g, f"come `{g.alias}` hai immesso un intero come richiesto", f"ovviamente durante lo svolgimento dell'esame non posso dirti se l'intero immesso sia poi la risposta corretta, ma il formato è corretto")            
        return True
                
    def verify_feasibility(self, SEF):
        return True
                
    def verify_consistency(self, SEF):
        return True
                
    def set_up_and_cash_handy_data(self):
        pass

    def verify_submission(self, SEF):
        if not self.verify_format(SEF):
            return SEF.completed_feedback
        self.set_up_and_cash_handy_data()
        if not self.verify_feasibility(SEF):
            return SEF.completed_feedback
        if not self.verify_consistency(SEF):
            return SEF.completed_feedback
        return SEF.feedback_when_all_checks_passed()
# extending class verify_submission_gen with ad-hoc virtual methods for problem knapsack

from typing import Optional, List, Dict, Tuple
from types import SimpleNamespace

class verify_submission_problem_specific(verify_submission_gen):
    def __init__(self, SEF,input_data_assigned:Dict, long_answer_dict:Dict):
        super().__init__(SEF,input_data_assigned, long_answer_dict)

    def verify_format(self, SEF):
        if not super().verify_format(SEF):
            return False
        if 'opt_val' in self.goals:
            g = self.goals['opt_val']
            if type(g.answ) != int:
                return SEF.format_NO(g, f"Come `{g.alias}` hai immesso `{g.answ}` dove era invece richiesto di immettere un intero.")
            SEF.format_OK(g, f"come `{g.alias}` hai immesso un intero come richiesto", f"ovviamente durante lo svolgimento dell'esame non posso dirti se l'intero immesso sia poi la risposta corretta, ma il formato è corretto")            
        if 'num_opt_sols' in self.goals:
            g = self.goals['num_opt_sols']
            if type(g.answ) != int:
                return SEF.format_NO(g, f"Come `{g.alias}` hai immesso `{g.answ}` dove era invece richiesto di immettere un intero.")
            SEF.format_OK(g, f"come `{g.alias}` hai immesso un intero come richiesto", f"ovviamente durante lo svolgimento dell'esame non posso dirti se l'intero immesso sia poi la risposta corretta, ma il formato è corretto")            
        if 'opt_sol' in self.goals:
            g = self.goals['opt_sol']
            if type(g.answ) != list:
                return SEF.format_NO(g, f"Come `{g.alias}` è richiesto si inserisca una lista di oggetti (esempio ['{self.I.labels[0]}','{self.I.labels[2]}']). Hai invece immesso `{g.answ}`.")
            for ele in g.answ:
                if ele not in self.I.labels:
                    return SEF.format_NO(g, f"Ogni oggetto che collochi nella lista `{g.alias}` deve essere uno degli elementi disponibili. L'elemento `{ele}` da tè inserito non è tra questi. Gli oggetti disponibili sono {self.I.labels}.")
            SEF.format_OK(g, f"come `{g.alias}` hai immesso un sottoinsieme degli oggetti dell'istanza originale", f"resta da stabilire l'ammissibilità di `{g.alias}`")
        return True
                
    def set_up_and_cash_handy_data(self):
        if 'opt_sol' in self.goals:
            self.sum_vals = sum([val for ele,cost,val in zip(self.I.labels,self.I.costs,self.I.vals) if ele in self.goals['opt_sol'].answ])
            self.sum_costs = sum([cost for ele,cost,val in zip(self.I.labels,self.I.costs,self.I.vals) if ele in self.goals['opt_sol'].answ])
            
    def verify_feasibility(self, SEF):
        if not super().verify_feasibility(SEF):
            return False
        if 'opt_sol' in self.goals:
            g = self.goals['opt_sol']
            for ele in g.answ:
                if ele in self.I.forced_out:
                    return SEF.feasibility_NO(g, f"L'oggetto `{ele}` da tè inserito nella lista `{g.alias}` è tra quelli proibiti. Gli oggetti proibiti per la Richiesta {str(SEF.task_number)}, sono {self.I.forced_out}.")
            for ele in self.I.forced_in:
                if ele not in g.answ:
                    return SEF.feasibility_NO(g, f"Nella lista `{g.alias}` hai dimenticato di inserire l'oggetto `{ele}` che invece è forzato. Gli oggetti forzati per la Richiesta {str(SEF.task_number)} sono {self.I.forced_in}.")
            if self.sum_costs > self.I.Knapsack_Capacity:
                return SEF.feasibility_NO(g, f"La tua soluzione in `{g.alias}` ha costo {self.sum_costs} > Knapsack_Capacity e quindi NON è ammissibile in quanto fora il budget per la Richiesta {str(SEF.task_number)}. La soluzione da tè inserita ricomprende il sottoinsieme di oggetti `{g.alias}`= {g.answ}.")
            SEF.feasibility_OK(g, f"come `{g.alias}` hai immesso un sottoinsieme degli oggetti dell'istanza originale", f"resta da stabilire l'ottimalità di `{g.alias}`")
        return True
                
    def verify_consistency(self, SEF):
        if not super().verify_consistency(SEF):
            return False
        if 'opt_val' in self.goals and 'opt_sol' in self.goals:
            g_val = self.goals['opt_val']; g_sol = self.goals['opt_sol'];
            if self.sum_vals != g_val.answ:
                return SEF.consistency_NO(['opt_val','opt_sol'], f"Il valore totale della soluzione immessa in `{g_sol.alias}` è {self.sum_vals}, non {g_val.answ} come hai invece immesso in `{g_val.alias}`. La soluzione (ammissibile) che hai immesso è `{g_sol.alias}`={g_sol.answ}.")
            SEF.consistency_OK(['opt_val','opt_sol'], f"{g_val.alias}={g_val.answ} = somma dei valori sugli oggetti in `{g_sol.alias}`.", "")
        return True


Si consideri uno zaino di capienza $CapacityMax$ = 36 e i seguenti oggetti ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'], aventi i seguenti costi [15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7] e valori [50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22].

__Richiesta 1 [40 punti]__: Tra i sottoinsiemi di oggetti di peso complessivo non eccedente $CapacityMax$= 36 fornirne uno in cui sia massima la somma dei valori.

In [ ]:
#Inserisci la tua risposta in forma di lista di oggetti da prendere (esempio: ['C','F','A'])
opt_sol1=[]
#Specificare il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi):
opt_val1=-1

In [ ]:
needed_varnames = ['opt_sol1', 'opt_val1', 'DPtable1', 'forced_out1', 'forced_in1']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 1, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':36,'forced_out':[],'forced_in':[],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol1,'opt_sol1'),'opt_val':(opt_val1,'opt_val1'),})

In [ ]:
loader_main()


__Richiesta 2 [40 punti]__: Tra i sottoinsiemi di oggetti di peso complessivo non eccedente <b>la capacità 32</b>, fornirne uno in cui sia massima la somma dei valori.

In [ ]:
#Inserisci in `opt_sol2` la lista degli oggetti da prendere (esempio: ['N', 'M', 'L', 'I', 'H', 'G', 'E', 'A'])
opt_sol2=[]
#Immetti in `opt_val2` il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi)
opt_val2=-1

In [ ]:
needed_varnames = ['opt_sol2', 'opt_val2', 'DPtable2', 'forced_out2', 'forced_in2']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 2, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':32,'forced_out':[],'forced_in':[],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol2,'opt_sol2'),'opt_val':(opt_val2,'opt_val2'),})

In [ ]:
loader_main()


__Richiesta 3 [40 punti]__: Tra i sottoinsiemi di oggetti di peso complessivo non eccedente <b>la capacità 30</b>, fornirne uno in cui sia massima la somma dei valori.

In [ ]:
#Inserisci in `opt_sol3` la lista degli oggetti da prendere (esempio: ['N', 'M', 'L', 'I', 'H', 'D', 'G'])
opt_sol3=[]
#Immetti in `opt_val3` il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi)
opt_val3=-1

In [ ]:
needed_varnames = ['opt_sol3', 'opt_val3', 'DPtable3', 'forced_out3', 'forced_in3']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 3, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':30,'forced_out':[],'forced_in':[],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol3,'opt_sol3'),'opt_val':(opt_val3,'opt_val3'),})

In [ ]:
loader_main()


__Richiesta 4 [40 punti]__: Tra i sottoinsiemi di oggetti di peso complessivo non eccedente <b>la capacità 28</b>, fornirne uno in cui sia massima la somma dei valori.

In [ ]:
#Inserisci in `opt_sol4` la lista degli oggetti da prendere (esempio: ['N', 'M', 'L', 'I', 'A', 'B'])
opt_sol4=[]
#Immetti in `opt_val4` il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi)
opt_val4=-1

In [ ]:
needed_varnames = ['opt_sol4', 'opt_val4', 'DPtable4', 'forced_out4', 'forced_in4']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 4, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':28,'forced_out':[],'forced_in':[],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol4,'opt_sol4'),'opt_val':(opt_val4,'opt_val4'),})

In [ ]:
loader_main()


__Richiesta 5 [40 punti]__: Fornire una soluzione ottima se <b>36 è la capienza dello zaino</b> da non superarsi ma assumendo di <b>non poter prendere</b> nessuno degli elementi in ['E'].

In [ ]:
#Inserisci in `opt_sol5` la lista degli oggetti da prendere (esempio: ['E', 'N', 'M', 'L', 'I', 'H'])
opt_sol5=[]
#Immetti in `opt_val5` il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi)
opt_val5=-1

In [ ]:
needed_varnames = ['opt_sol5', 'opt_val5', 'DPtable5', 'forced_out5', 'forced_in5']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 5, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':36,'forced_out':['E'],'forced_in':[],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol5,'opt_sol5'),'opt_val':(opt_val5,'opt_val5'),})

In [ ]:
loader_main()


__Richiesta 6 [40 punti]__: Fornire una soluzione ottima se <b>36 è la capienza dello zaino</b> da non superarsi ma assumendo di <b>non poter prendere</b> nessuno degli elementi in ['B', 'E'].

In [ ]:
#Inserisci in `opt_sol6` la lista degli oggetti da prendere (esempio: ['E', 'N', 'M', 'L', 'C', 'A'])
opt_sol6=[]
#Immetti in `opt_val6` il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi)
opt_val6=-1

In [ ]:
needed_varnames = ['opt_sol6', 'opt_val6', 'DPtable6', 'forced_out6', 'forced_in6']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 6, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':36,'forced_out':['B', 'E'],'forced_in':[],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol6,'opt_sol6'),'opt_val':(opt_val6,'opt_val6'),})

In [ ]:
loader_main()


__Richiesta 7 [40 punti]__: Fornire una soluzione ottima se <b>34 è la capienza dello zaino</b> da non superarsi ma assumendo di <b>non poter prendere</b> nessuno degli elementi in ['B', 'E', 'F'].

In [ ]:
#Inserisci in `opt_sol7` la lista degli oggetti da prendere (esempio: ['F', 'N', 'M', 'L', 'I', 'H', 'D', 'C'])
opt_sol7=[]
#Immetti in `opt_val7` il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi)
opt_val7=-1

In [ ]:
needed_varnames = ['opt_sol7', 'opt_val7', 'DPtable7', 'forced_out7', 'forced_in7']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 7, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':34,'forced_out':['B', 'E', 'F'],'forced_in':[],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol7,'opt_sol7'),'opt_val':(opt_val7,'opt_val7'),})

In [ ]:
loader_main()


__Richiesta 8 [40 punti]__: Fornire una soluzione ottima se <b>34 è la capienza dello zaino</b> da non superarsi ma assumendo di <b>dover prendere</b> tutti gli elementi in ['B', 'E'].

In [ ]:
#Inserisci in `opt_val8` il massimo valore possibile per una soluzione ammissibile (un intero, la somma dei valori degli oggetti presi)
opt_val8=-1

In [ ]:
needed_varnames = ['opt_sol8', 'opt_val8', 'DPtable8', 'forced_out8', 'forced_in8']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 8, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':34,'forced_out':[],'forced_in':['B', 'E'],'partialDPtable':[],}, long_answer_dict={'opt_val':(opt_val8,'opt_val8'),})

In [ ]:
loader_main()


__Richiesta 9 [40 punti]__: Fornire una soluzione ottima se <b>34 è la capienza dello zaino</b> da non superarsi ma assumendo di <b>dover prendere tutti</b> gli elementi in ['B', 'F'] e <b>nessuno</b> di quelli in ['E'].

In [ ]:
#Inserisci in `opt_sol9` la lista degli oggetti da prendere (esempio: ['F', 'E', 'N', 'M', 'L', 'G', 'C'])
opt_sol9=[]
#Immetti in `opt_val9` il valore della soluzione introdotta (un intero, la somma dei valori degli oggetti presi)
opt_val9=-1

In [ ]:
needed_varnames = ['opt_sol9', 'opt_val9', 'DPtable9', 'forced_out9', 'forced_in9']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 9, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':34,'forced_out':['E'],'forced_in':['B', 'F'],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol9,'opt_sol9'),'opt_val':(opt_val9,'opt_val9'),})

In [ ]:
loader_main()


__Richiesta 10 [40 punti]__: Fornire una soluzione ottima se <b>31 è la capienza dello zaino</b> da non superarsi ma assumendo di <b>dover prendere tutti</b> gli elementi in ['B', 'I'] e <b>nessuno</b> di quelli in ['F', 'E'].

In [ ]:
#Inserisci la tua risposta in forma di lista di oggetti da prendere (esempio: ['I', 'E', 'N', 'M', 'L', 'F', 'D'])
opt_sol10=[]

In [ ]:
needed_varnames = ['opt_sol10', 'opt_val10', 'DPtable10', 'forced_out10', 'forced_in10']
for varname in needed_varnames:
    if varname not in locals():
        exec(f'{varname} = None')
# launch all requests for evaluation of the answer objects and get the feedback

def check_checkers_agreement(URL_1_NAME, TAL_feedback_dict1, feedback_dict2, URL_2_NAME=None):
    if URL_2_NAME is None:
        term_of_comparison = "that of the local King Arthur placed in the notebook"
    else:
        term_of_comparison = "that from server {URL_2_NAME}"
    if TAL_feedback_dict1 != None and feedback_dict2 != None and 'pt_safe' in TAL_feedback_dict1 and 'pt_maybe' in TAL_feedback_dict1 and 'pt_out' in TAL_feedback_dict1 and 'pt_safe' in feedback_dict2:
        if TAL_feedback_dict1['pt_safe'] != feedback_dict2['pt_safe']:
            print(f"\n\nWARNING: the TALight opinion (pt_safe={TAL_feedback_dict1['pt_safe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_safe={feedback_dict2['pt_safe']})")
        if TAL_feedback_dict1['pt_maybe'] != feedback_dict2['pt_maybe']:
            print(f"\n\nWARNING: the TALight opinion (pt_maybe={TAL_feedback_dict1['pt_maybe']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_maybe={feedback_dict2['pt_maybe']})")
        if TAL_feedback_dict1['pt_out'] != feedback_dict2['pt_out']:
            print(f"\n\nWARNING: the TALight opinion (pt_out={TAL_feedback_dict1['pt_out']}) from server `{URL_1_NAME}` differs from {term_of_comparison} (pt_out={feedback_dict2['pt_out']})")


def verify_submission(TALight_problem_name:str,checkers:List, task_dict:Dict, input_data_assigned:Dict, long_answer_dict:Dict):
    #print("\n\n\n" f"task_dict={task_dict}" "\n\n\n", file=stderr)
    task_dict['with_positive_enforcement'] = '1' if 'positive_enforcements' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['with_notes'] = '1' if 'cautious_notes' in settings['FEEDBACKS_TO_STUDENT'] else '0'
    task_dict['as_yaml_with_points'] = '1'
    task_dict['color_implementation'] = 'html'
    task_dict['with_output_files'] = '0'
    if settings['CHECKERS_INVOLVED'] == 'ONLY_THE_ONE_EMBEDDED_IN_NOTEBOOK':
        checkers=['embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_THE_TALIGHT_ONES':
        checkers=['TALight']
    elif settings['CHECKERS_INVOLVED'] == 'BOTH':
        checkers=['TALight', 'embedded_in_notebook']
    elif settings['CHECKERS_INVOLVED'] == 'ONLY_FIRST_IN_LIST_FROM_YAML_LONGFILE':
        checkers=checkers[0]
    elif settings['CHECKERS_INVOLVED'] != 'THOSE_IN_LIST_FROM_YAML_LONGFILE':
        print(f"Unknown option `{settings['CHECKERS_INVOLVED']}` for the setting `CHECKERS_INVOLVED`")
        return
    if 'TALight' in checkers:
        if len(settings['RTAL_URLS']) == 0:
            print("Invalid Settings File: `CHECKERS_INVOLVED`=`{settings['CHECKERS_INVOLVED']}` is in contraddiction with `RTAL_URLS` set to the empty list")
            return
        answer_dict = { val[1]:val[0] for val in long_answer_dict.values() }
        alias_dict = { val[1]:key for key, val in long_answer_dict.items() }
        rtalargs_dict = task_dict
        rtalargs_dict['input_data_assigned'] = input_data_assigned
        rtalargs_dict['alias_dict'] = alias_dict
        rtalargs_dict['answer_dict'] = answer_dict
        URL_NAME = settings['RTAL_URLS'][0]
        URL = settings[URL_NAME]
        TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
        monitor_what_submitted_to_rtald(TALight_problem_name,rtalargs_dict)
        print(f"Here is what the TALight checker thinks (and has stored on the server `{URL_NAME}`) about your submission:")
        if TAL_feedback_dict != None:
            display(TAL_feedback_dict['feedback_string'])
        elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
            print("The Feedback received from the server is EMPTY")
        for next_URL_NAME in settings['RTAL_URLS'][1:]:
            URL = settings[next_URL_NAME]
            print(f"Next we pose the very same request also to the server `{next_URL_NAME}`:")
            next_TAL_feedback_dict = rtal_connect(URL, TALight_problem_name, 'check', rtalargs_dict=rtalargs_dict, output_files_local_folder='output_files_TALight', rtaltoken=RTAL_PRIVATE_ACCESS_TOKEN)
            print(f"Here is what the TALight checker thinks (and has stored on the server `{next_URL_NAME}`) about your submission:")
            if next_TAL_feedback_dict != None:
                display(next_TAL_feedback_dict['feedback_string'])
                check_checkers_agreement(URL_NAME,TAL_feedback_dict,next_TAL_feedback_dict,next_URL_NAME)
            if next_TAL_feedback_dict != None:
                display(TAL_feedback_dict['feedback_string'])
            elif 'signal_when_no_feedback_has_been_produced' in setting['RTAL_INTERFACE_VERBOSITY']:
                print("The Feedback received from the server is EMPTY")

    if 'embedded_in_notebook' in checkers:
        print("Here is an independent evaluation (just in case the TALight server is down or unreachable):")
        SEF = std_eval_feedback(task_dict)
        KingArthur = verify_submission_problem_specific(SEF, input_data_assigned, long_answer_dict)
        embedded_feedback_dict = KingArthur.verify_submission(SEF)
        #print(f"feedback_dict={feedback_dict}", file=stderr)
        display(HTML(embedded_feedback_dict['feedback_string']))
        if 'TALight' in checkers and settings['CHECK_AGREEMENT_OF_CHECKERS']:
            check_checkers_agreement(URL_NAME,TAL_feedback_dict,embedded_feedback_dict)
verify_submission(TALight_problem_name='RO_knapsack',checkers=['TALight', 'embedded_in_notebook'],task_dict={'task': 10, 'pt_tot': 40, 'pt_formato_OK': 0, 'pt_feasibility_OK': 1, 'pt_consistency_OK': 0}, input_data_assigned={'labels':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N'],'costs':[15, 16, 17, 11, 13, 5, 7, 3, 1, 12, 9, 7],'vals':[50, 52, 54, 40, 45, 17, 18, 7, 8, 42, 30, 22],'Knapsack_Capacity':31,'forced_out':['F', 'E'],'forced_in':['B', 'I'],'partialDPtable':[],}, long_answer_dict={'opt_sol':(opt_sol10,'opt_sol10'),})

In [ ]:
loader_main()


<h1>_______________________________________________________________________________________ </h1>

In [ ]:
import os
import time
from urllib.request import urlopen
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert import RSTExporter, NotebookExporter
from IPython.display import Image, HTML, Javascript
from datetime import datetime

import shutil
last ='preview_last/' #path of last submission
old = 'preview_old/' #path of prevous submissions

#hide alert buttons
def hide_w(w):
    for e in w:
        e.layout.visibility = 'hidden'

#show alert buttons
def show_w(w):
    for e in w:
        e.layout.visibility = 'visible'

def are_you_sure():
    show_w(alert_buttons)

label = widgets.Label(value="Un tuo elaborato era gi stato precedentemente prodotto. Vuoi rimpiazzarlo con il presente?")
yes = widgets.Button(description="Si", tooltip="Produce il nuovo elaborato. Esso diviene la versione attuale prendendo il posto del precedente")
no = widgets.Button(description="No")

alert_buttons = [label,yes,no]
hide_w(alert_buttons)

#move all files from last to old and save the new preview
def yes_clicked(b):
    files = os.listdir(last)
    for f in files:
        if f == '.ipynb_checkpoints':
            continue
        if not os.path.isdir('./preview_old'):
            os.mkdir(old)
        shutil.move(last+f, old)
    display(Javascript('IPython.notebook.save_checkpoint();'))
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
    hname = 'Esercizio_1.ipynb'[:-6] + '_' + date_time
    ! jupyter nbconvert Esercizio_1.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['noexport']"
    display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    hide_w(alert_buttons)
def no_clicked(b):
    hide_w(alert_buttons)


def generate_preview_HTML(_):
    if not os.path.isdir('./preview_last'):
        os.mkdir(last)
    directory= os.listdir(last)
    if len(directory) <1:
        display(Javascript('IPython.notebook.save_checkpoint();'))
        now = datetime.now() # current date and time
        date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
        hname = 'Esercizio_1.ipynb'[:-6] + '_' + date_time
        ! jupyter nbconvert Esercizio_1.ipynb --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['noexport']" --to html_embed --output=$hname --output-dir=./preview_last/
        display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    else:
        are_you_sure()


button = widgets.Button(description="Salva & Esporta", tooltip="Esporta il foglio Jupyter in HTML nella cartella preview")
output = widgets.Output()
button.on_click(generate_preview_HTML)
yes.on_click(yes_clicked)
no.on_click(no_clicked)
h_box1 = widgets.HBox([button])
h_box2 = widgets.HBox([label,yes,no])
display(widgets.VBox([h_box1,h_box2]))
